<h1>Walmart Sales Forecasting-- Feature Engineering </h1>

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import multiprocessing as mp
import gc
import datetime
from sklearn.preprocessing import LabelEncoder
import calendar
from scipy.sparse import csr_matrix,hstack
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from tqdm import tqdm
import pickle

In [2]:
#Reading up the dataframes
train=pd.read_csv('final_dataframe.csv')
test=pd.read_csv('final_dataframe_test.csv')
final_test=pd.read_csv('final_future_data.csv')

<h1>Reducing Memory by converting all categorical varaibles to integer</h1>

In [3]:
# Now here we are also saving these label encoders data so that we can use them to encode for unkown data in future

In [7]:
lbl=LabelEncoder()
train['item_id']=lbl.fit_transform(train['item_id'])
test['item_id']=lbl.transform(test['item_id'])
final_test['item_id']=lbl.transform(final_test['item_id'])
pickle.dump(lbl,open('label_encoder_item_id.sav','wb'))

In [8]:
lbl=LabelEncoder()
train['dept_id']=lbl.fit_transform(train['dept_id'])
test['dept_id']=lbl.transform(test['dept_id'])
final_test['dept_id']=lbl.transform(final_test['dept_id'])
pickle.dump(lbl,open('label_encoder_dept_id.sav','wb'))

In [9]:
lbl=LabelEncoder()
train['cat_id']=lbl.fit_transform(train['cat_id'])
test['cat_id']=lbl.transform(test['cat_id'])
final_test['cat_id']=lbl.transform(final_test['cat_id'])
pickle.dump(lbl,open('label_encoder_cat_id.sav','wb'))

In [10]:
lbl=LabelEncoder()
train['store_id']=lbl.fit_transform(train['store_id'])
test['store_id']=lbl.transform(test['store_id'])
final_test['store_id']=lbl.transform(final_test['store_id'])
pickle.dump(lbl,open('label_encoder_store_id.sav','wb'))

In [11]:
lbl=LabelEncoder()
train['state_id']=lbl.fit_transform(train['state_id'])
test['state_id']=lbl.transform(test['state_id'])
final_test['state_id']=lbl.transform(final_test['state_id'])
pickle.dump(lbl,open('label_encoder_state_id.sav','wb'))

In [12]:
lbl=LabelEncoder()
train['event_name_1']=lbl.fit_transform(train['event_name_1'])
test['event_name_1']=lbl.transform(test['event_name_1'])
final_test['event_name_1']=lbl.transform(final_test['event_name_1'])
pickle.dump(lbl,open('label_encoder_event_name_1.sav','wb'))

In [13]:
lbl=LabelEncoder()
train['event_name_2']=lbl.fit_transform(train['event_name_2'])
test['event_name_2']=lbl.transform(test['event_name_2'])
final_test['event_name_2']=lbl.transform(final_test['event_name_2'])
pickle.dump(lbl,open('label_encoder_event_name_2.sav','wb'))

In [14]:
lbl=LabelEncoder()
train['event_type_1']=lbl.fit_transform(train['event_type_1'])
test['event_type_1']=lbl.transform(test['event_type_1'])
final_test['event_type_1']=lbl.transform(final_test['event_type_1'])
pickle.dump(lbl,open('label_encoder_event_type_1.sav','wb'))

In [15]:
lbl=LabelEncoder()
train['event_type_2']=lbl.fit_transform(train['event_type_2'])
test['event_type_2']=lbl.transform(test['event_type_2'])
final_test['event_type_2']=lbl.transform(final_test['event_type_2'])
pickle.dump(lbl,open('label_encoder_event_type_2.sav','wb'))

In [16]:
lbl=LabelEncoder()
train['year']=lbl.fit_transform(train['year'])
test['year']=lbl.transform(test['year'])
final_test['year']=lbl.transform(final_test['year'])
pickle.dump(lbl,open('label_encoder_year.sav','wb'))

<h3>Removing Unnecessary columns</h3>

In [4]:
%%time
# Converting snap_CA,snap_WI,snap_TX into one feature named snap
train.loc[train['state_id'] == 'CA', 'snap'] = train.loc[train['state_id'] == 'CA']['snap_CA']
train.loc[train['state_id'] == 'TX', 'snap'] = train.loc[train['state_id'] == 'TX']['snap_TX']
train.loc[train['state_id'] == 'WI', 'snap'] = train.loc[train['state_id'] == 'WI']['snap_WI']
train.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)


test.loc[test['state_id'] == 'CA', 'snap'] = test.loc[test['state_id'] == 'CA']['snap_CA']
test.loc[test['state_id'] == 'TX', 'snap'] = test.loc[test['state_id'] == 'TX']['snap_TX']
test.loc[test['state_id'] == 'WI', 'snap'] = test.loc[test['state_id'] == 'WI']['snap_WI']
test.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)

final_test.loc[final_test['state_id'] == 'CA', 'snap'] = final_test.loc[final_test['state_id'] == 'CA']['snap_CA']
final_test.loc[final_test['state_id'] == 'TX', 'snap'] = final_test.loc[final_test['state_id'] == 'TX']['snap_TX']
final_test.loc[final_test['state_id'] == 'WI', 'snap'] = final_test.loc[final_test['state_id'] == 'WI']['snap_WI']
final_test.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)

CPU times: user 29 s, sys: 3.23 s, total: 32.3 s
Wall time: 32.2 s


In [5]:
%%time
# Weekday as wday are similar features so we remove it
#I will make no use of  wm_yr_wk feature
train.drop('weekday',axis=1,inplace=True)
train.drop('wm_yr_wk',axis=1,inplace=True)
 
test.drop('weekday',axis=1,inplace=True)
test.drop('wm_yr_wk',axis=1,inplace=True)

final_test.drop('weekday',axis=1,inplace=True)
final_test.drop('wm_yr_wk',axis=1,inplace=True)

CPU times: user 8.64 s, sys: 3.6 s, total: 12.2 s
Wall time: 12.2 s


<h2>1.Date Related Features</h2>

<h3>1.Week Number</h3>

In [21]:
#Reference https://stackoverflow.com/questions/2600775/how-to-get-week-number-in-python/32638267#:~:text=()%5B1%5D%2024-,datetime.,for%20the%20given%20date%20instance.&text=You%20can%20get%20the%20week%20number%20directly%20from%20datetime%20as%20string.
def get_week_number(x):
    """This Function is used to get weeknumber of particular date"""
    date=calendar.datetime.date.fromisoformat(x)
    return date.isocalendar()[1]

In [22]:
train['week_number']=train['date'].apply(lambda x:get_week_number(x))
test['week_number']=test['date'].apply(lambda x:get_week_number(x))
final_test['week_number']=final_test['date'].apply(lambda x:get_week_number(x))

<h3>2.Season</h3>

In [23]:
#https://www.universaltraveller.com.au/destinations/los-angeles/weather
def get_season(x):
    """This function is used to get season in US according to various months"""
    if x in [12,1,2]:
        return 0      #"Winter"
    elif x in [3,4,5]:
        return 1   #"Spring"
    elif x in [6,7,8]:
        return 2   #"Summer"
    else:
        return 3   #"Autumn"

In [24]:
train['season']=train['month'].apply(lambda x:get_season(x))
test['season']=test['month'].apply(lambda x:get_season(x))
final_test['season']=final_test['month'].apply(lambda x:get_season(x))

<h3>3.Quater Start</h3>

In [25]:
def check_if_quater_begin(x):
    """This is used to check if day is begining of quater"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and (month in [1,4,7,9])) else 0

In [26]:
train['quater_start']=train['date'].apply(lambda x:check_if_quater_begin(x))
test['quater_start']=test['date'].apply(lambda x:check_if_quater_begin(x))
final_test['quater_start']=final_test['date'].apply(lambda x:check_if_quater_begin(x))

<h3>4.Quater End</h3>

In [27]:
#Reference https://www.lawinsider.com/dictionary/quarter-end
def check_if_quater_end(x):
    """This is used to check if day is end of quater"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    if (day==31 and month==3) or (day==30 and month==6) or (day==30 and month==9) or (day==31 and month==12):
        return 1
    else:
        return 0

In [28]:
train['quater_end']=train['date'].apply(lambda x:check_if_quater_end(x))
test['quater_end']=test['date'].apply(lambda x:check_if_quater_end(x))
final_test['quater_end']=final_test['date'].apply(lambda x:check_if_quater_end(x))

<h3>5.Month Start</h3>

In [29]:
def month_start(x):
    """This is used to check if day is begining of month"""
    day=calendar.datetime.date.fromisoformat(x).day
    return 1 if day==1 else 0

In [30]:
train['month_start']=train['date'].apply(lambda x:month_start(x))
test['month_start']=test['date'].apply(lambda x:month_start(x))
final_test['month_start']=final_test['date'].apply(lambda x:month_start(x))

<h3>6.Month End</h3>

In [31]:
def month_end(x):
    """This is used to check if day is end of month"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    year=calendar.datetime.date.fromisoformat(x).year
    leap_yr=(year%4==0) # to check if it is a leap year
    val=(day==31 and month==1) or (day==29 if leap_yr else day==28) or (day==31 and month==3) or (day==30 and month==4) or\
        (day==31 and month==5) or (day==30 and month==6) or (day==31 and month==7) or (day==31 and month==8) or\
        (day==30 and month==9) or (day==31 and month==10) or (day==30 and month==11) or (day==31 and month==12)
    return 1 if val else 0

In [32]:
train['month_end']=train['date'].apply(lambda x:month_end(x))
test['month_end']=test['date'].apply(lambda x:month_end(x))
final_test['month_end']=final_test['date'].apply(lambda x:month_end(x))

<h3>7.Year Start</h3>

In [33]:
def year_start(x):
    """This is used to check if day is begining of year"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and month==1) else 0

In [34]:
train['year_start']=train['date'].apply(lambda x:year_start(x))
test['year_start']=test['date'].apply(lambda x:year_start(x))
final_test['year_start']=final_test['date'].apply(lambda x:year_start(x))

<h3>8.Year End</h3>

In [35]:
def year_end(x):
    """This is used to check if day is end of year"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==31 and month==12) else 0

In [36]:
train['year_end']=train['date'].apply(lambda x:year_end(x))
test['year_end']=test['date'].apply(lambda x:year_end(x))
final_test['year_end']=final_test['date'].apply(lambda x:year_end(x))

<h2> Taking Last 28 days from train as CrossValidation so that it can be used while Modeling</h2>

In [37]:
#Cross Validation data will be used for hyperparameter tuning
cv=train[train['date']>='2016-03-28']
train=train[train['date']<'2016-03-28']

<h2> Time Series Related Features </h2>

In [7]:
%%time
#Firstly we will create these Direct features for train  and CV test and final test data
# Code to create one large data for all days
gc.collect()
tt=pd.concat([train,cv,test,final_test])
tt.sort_values(['id','date'],inplace=True)
df=tt.pivot_table(index=['item_id','store_id'],columns='date',values='sales')
df.fillna(0,inplace=True)

CPU times: user 1min 10s, sys: 17.7 s, total: 1min 28s
Wall time: 1min 28s


<h3> 1.Rolling mean and Rolling Standard Deviation </h3>

In [10]:
%%time
#this code snippet is used to calculate rolling mean and rolling std of sales data with shift of 28 days 
# Here we are taking 28 days shift so as to avoid Data Leakage Problem
for aggregate in ['mean','std']:
    for shif in [28]:
        for r in [7,14,30,60,360]:
            roll=df.rolling(r,axis=1).agg(aggregate).shift(shif)
            dates=roll.columns
            name="roll_"+str(r)+"_shift_"+str(shif)+"_"+aggregate
            roll=roll.astype('float16')
            roll.reset_index(level=[0,1],inplace=True)
            roll=pd.melt(roll,id_vars=['item_id','store_id'],value_vars=dates,var_name='date',value_name=name)
            roll.fillna(-1,inplace=True)
            train=train.merge(roll,on=['item_id','store_id','date'])
            cv=cv.merge(roll,on=['item_id','store_id','date'])
            final_test=final_test.merge(roll,on=['item_id','store_id','date'])
            test=test.merge(roll,on=['item_id','store_id','date'])
            print("Feature created named :=",name)
            del roll
            gc.collect()

Feature created named := roll_7_shift_28_mean
Feature created named := roll_14_shift_28_mean
Feature created named := roll_30_shift_28_mean
Feature created named := roll_60_shift_28_mean
Feature created named := roll_360_shift_28_mean
Feature created named := roll_7_shift_28_std
Feature created named := roll_14_shift_28_std
Feature created named := roll_30_shift_28_std
Feature created named := roll_60_shift_28_std
Feature created named := roll_360_shift_28_std
CPU times: user 13min 29s, sys: 3min 2s, total: 16min 31s
Wall time: 16min 32s


<h3>2. Exponential Weighted Average</h3>

In [11]:
# Adding  Exponential weighted average with shift of 28 days
# Shift of 28 days is used to prevent data leakage Problem
roll=df.shift(28,axis=1).ewm(alpha=0.99,axis=1,adjust=False).mean()
dates=roll.columns
roll=roll.astype('float16')
roll.reset_index(level=[0,1],inplace=True)
roll=pd.melt(roll,id_vars=['item_id','store_id'],value_vars=dates,var_name='date',value_name='direct_ewm')
roll.fillna(-1,inplace=True)
train=train.merge(roll,on=['item_id','store_id','date'])
cv=cv.merge(roll,on=['item_id','store_id','date'])
test=test.merge(roll,on=['item_id','store_id','date'])
final_test=final_test.merge(roll,on=['item_id','store_id','date'])
print("Direct Feature created ewa window of size")

Direct Feature created ewa window of size


<h3>3. Lag Features</h3>

In [12]:
%%time
# Now we will also calculate lag features with lag of 28,35,42,49,56,63,70,77,84,91,96 days
for lag in range(28,100,7):
    i='direct_lag_'+str(lag)
    lag_i=df.shift(lag,axis=1)
    dates=lag_i.columns
    lag_i.reset_index(level=[0,1],inplace=True)
    lag_i=pd.melt(lag_i,id_vars=['item_id','store_id'],value_vars=dates,var_name='date',value_name=i)
    lag_i.fillna(-1,inplace=True)
    lag_i[i]=lag_i[i].astype('int16')
    train=train.merge(lag_i,on=['item_id','store_id','date'])
    cv=cv.merge(lag_i,on=['item_id','store_id','date'])
    test=test.merge(lag_i,on=['item_id','store_id','date'])
    final_test=final_test.merge(lag_i,on=['item_id','store_id','date'])
    print("Feature created for lag",lag)
    del lag_i
    gc.collect()

Feature created for lag 28
Feature created for lag 35
Feature created for lag 42
Feature created for lag 49
Feature created for lag 56
Feature created for lag 63
Feature created for lag 70
Feature created for lag 77
Feature created for lag 84
Feature created for lag 91
Feature created for lag 98
CPU times: user 14min 56s, sys: 3min 22s, total: 18min 18s
Wall time: 18min 19s


<h2> Saving All features Created </h2>

In [39]:
%%time
train.to_csv('train1.csv',index=False)
cv.to_csv('cv1.csv',index=False)
test.to_csv('test1.csv',index=False)
final_test.to_csv('final_test1.csv',index=False)

CPU times: user 43.1 s, sys: 696 ms, total: 43.8 s
Wall time: 46.1 s


<b>Some Points</b>
1. Here we have constructed all time series Related features(lag/rolls) with shift of 28 days bcz we may not get stuck into data leakage Problem.